In [26]:
import codecs
import operator
import re
import nltk
import gensim
num_regex = re.compile(r'^[+-]?[0-9]+\.?[0-9]*$')

In [27]:
def is_number(token):
    return bool(num_regex.match(token))

In [28]:
def build_vocab(maxlen=0, vocab_size=0):
    
    source = './' + 'train.txt'

    total_words, unique_words = 0, 0
    word_freqs = {}
    #W_to_Vmodel = gensim.models.Word2Vec.load("w2v_embedding")
    fin = codecs.open(source, 'r', 'utf-8')
    #input_file=open(source,'r')
    #fin=input_file.readlines()
    for line in fin:
        words = line.split()
		#print(words)
        #line = line.lower()
        #stop_words = ['government','country','state','district','rajya','sabha']
        #resultwords  = [word for word in re.split("\W+",line) if word.lower() not in stop_words]
        #line = ' '.join(resultwords)
        #is_noun = lambda pos: pos[:2] == 'NN'
        #tokenized = nltk.word_tokenize(line)
        #nltk.download('averaged_perceptron_tagger')
        #words = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
        #words = str(nouns)
        if maxlen > 0 and len(words) > maxlen:
            continue

        for w in words:
            if not is_number(w):
                try:
                    word_freqs[w] += 1
                except KeyError:
                    unique_words += 1
                    word_freqs[w] = 1
                total_words += 1

    print(' Reader ::    %i total words, %i unique words' % (total_words, unique_words))
    sorted_word_freqs = sorted(word_freqs.items(), key=operator.itemgetter(1), reverse=True)

    vocab = {'<pad>': 0, '<unk>': 1, '<num>': 2}
    index = len(vocab)

    for word, _ in sorted_word_freqs:
        vocab[word] = index
        index += 1
        if vocab_size > 0 and index > vocab_size + 2:
            break
    if vocab_size > 0:
        print(' Reader ::   keep the top %i words' % vocab_size)

    # Write (vocab, frequence) to a txt file
#     vocab_file = codecs.open('./vocab', mode='w', encoding='utf8')
#     sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
#     for word, index in sorted_vocab:
#         if word in W_to_Vmodel.wv.vocab:
#             if index < 3:
#                 vocab_file.write(word + '\t' + str(0) + '\n')
#                 continue
#             vocab_file.write(word + '\t' + str(word_freqs[word]) + '\n')
#     vocab_file.close()

    return vocab

In [29]:
def read_set(phase, vocab, maxlen):
    
    #assert phase in {'train', 'test'}

    source = './' + phase + '.txt'
    num_hit, unk_hit, total = 0., 0., 0.
    maxlen_x = 0
    data_x = []

    fin = codecs.open(source, 'r', 'utf-8')
    for line in fin:
        line=line.lower()
        stop_words=['']
        words = line.strip().split()
        if maxlen > 0 and len(words) > maxlen:
            continue
        if not len(words):
            continue

        indices = []
        for word in words:
            if is_number(word):
                indices.append(vocab['<num>'])
                num_hit += 1
            elif word in vocab:
                indices.append(vocab[word])
            else:
                indices.append(vocab['<unk>'])
                unk_hit += 1
            total += 1

        data_x.append(indices)
        if maxlen_x < len(indices):
            maxlen_x = len(indices)

    print(' Reader ::    <num> hit rate: %.2f%%, <unk> hit rate: %.2f%%' % (100 * num_hit / total, 100 * unk_hit / total))
    return data_x, maxlen_x

In [30]:
def fetch_data(vocab_size=0, maxlen=0):
    print(' Reader :: Reading data from training set')
    print(' Reader ::  Creating vocab ...')
    vocab = build_vocab(maxlen, vocab_size)
    print(' Reader ::  Reading dataset ...')
    print(' Reader ::   train set')
    train_x, train_maxlen = read_set('train', vocab, maxlen)
    print(' Reader ::   test set')
    test_x, test_maxlen = read_set('test', vocab, maxlen)
    maxlen = max(train_maxlen, test_maxlen)
    print(train_maxlen)
    print(test_maxlen)
    return vocab, train_x, test_x, maxlen